In [133]:
import numpy as np
from scipy.special import logsumexp
from scipy.integrate import solve_ivp, odeint
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt
from equilibrator_api import ComponentContribution, Q_, Reaction
import requests
import xmltodict
import pint
import cvxpy as cp
import itertools
import copy
from convex_kinetics_new import ConvexKineticsNew

sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

A + B -E1> C -E2> D

B -E3> F

A -E4> G

# Testing eQuilibrator

In [2]:
cc = ComponentContribution()

# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.0)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("298.15K")



In [3]:
from equilibrator_api import Reaction
compound_ids = ["WATER", "ADP", "ATP", "Pi"]
compound_dict = {cid : cc.get_compound(f"metacyc.compound:{cid}") for cid in compound_ids}
atpase_reaction = Reaction({
    compound_dict["ATP"]: -1,
    compound_dict["WATER"]: -1,
    compound_dict["ADP"]: 1,
    compound_dict["Pi"]: 1,
})

In [4]:
standard_dg_prime = cc.standard_dg_prime(atpase_reaction)
standard_dg_prime

In [5]:
cytoplasmic_p_h = Q_(7.5)
cytoplasmic_ionic_strength = Q_("250 mM")
periplasmic_p_h = Q_(7.0)
periplasmic_ionic_strength = Q_("200 mM")
e_potential_difference = Q_("0.15 V")
cytoplasmic_reaction = "bigg.metabolite:pep = bigg.metabolite:g6p + bigg.metabolite:pyr"
periplasmic_reaction = "bigg.metabolite:glc__D = "

cc = ComponentContribution()
cc.p_h = cytoplasmic_p_h
cc.ionic_strength = cytoplasmic_ionic_strength
standard_dg_prime = cc.multicompartmental_standard_dg_prime(
    cc.parse_reaction_formula(cytoplasmic_reaction),
    cc.parse_reaction_formula(periplasmic_reaction),
    e_potential_difference=e_potential_difference,
    p_h_outer=periplasmic_p_h,
    ionic_strength_outer=periplasmic_ionic_strength,
)

print(standard_dg_prime)


In [6]:
cc.get_compound_by_inchi("WQZGKKKJIJFFOK-GASJEMHNSA-N")

# Using eQuilibrator to generate equilibrium constants

In [9]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'#'})
# Issue web service request:
r = s.get('https://websvc.biocyc.org/getxml?id=ECOLI:6PFRUCTPHOS-RXN&detail=low&fmt=json')

In [10]:
name = 'F16ALDOLASE-RXN'

r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:{name}&detail=low&fmt=json')
o = xmltodict.parse(r.content)['ptools-xml']
o['Reaction']['enzymatic-reaction']['Enzymatic-Reaction'][0]['@frameid']

In [11]:
rxns_names = ['ABC-63-RXN', 'RXN0-5205']
# rxns_names = ['6PFRUCTPHOS-RXN', 'F16ALDOLASE-RXN', '2TRANSKETO-RXN', 'TRIOSEPISOMERIZATION-RXN']

rxns_dict = {}
stoich_dict = {}
regulation_dict = {}

for name in rxns_names:
    r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:{name}&detail=low&fmt=json')
    rxn = xmltodict.parse(r.content)['ptools-xml']

    rxn_dict = {}
    stoich_loop_dict = {}
    regulation_loop_dict = {}
    left = rxn['Reaction']['left']
    right = rxn['Reaction']['right'] 
    
    # i will know exact complex in model (connected to rxn), but for now will pick random
    enz_rxn = rxn['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
    
    if type(left) is dict:
        left = [left]
    
    if type(right) is dict:
        right = [right]
        
    if type(enz_rxn) is dict:
        enz_rxn = [enz_rxn]
    
    enz_rxn_id =  enz_rxn[0]['@frameid']
    re = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:{enz_rxn_id}&detail=high&fmt=json')
    oe = xmltodict.parse(re.content)['ptools-xml']['Enzymatic-Reaction']
    
    if 'regulated-by' in oe:
        oe = oe['regulated-by']['Regulation']
    
        if type(oe) is dict:
            oe = [oe]
            
        # add regulators, target regulator directly with id to get info like irreversible, etc
        for regulator in oe:
            reg_name = regulator['regulator']['Compound']['@frameid']
            reg_type = regulator['mode']['#text']
            reg_type = 1 if reg_type == '+' else -1
            regulation_loop_dict[reg_name] = reg_type
    
    for mol in left:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            
            compartment = mol['compartment']['cco']['@frameid']
            
            if compartment == 'CCO-IN':
                # mol_cc = mol_cc.in_compartment('c')
                compartment = 'c'
            elif compartment == 'CCO-OUT':
                # mol_cc = mol_cc.in_compartment('p')
                compartment = 'p'
            
            rxn_dict[mol_cc] = -1
            stoich_loop_dict[f'{cid}[{compartment}]'] = -1

    for mol in right:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            
            compartment = mol['compartment']['cco']['@frameid']
            
            if compartment == 'CCO-IN':
                # mol_cc = mol_cc.in_compartment('c')
                compartment = 'c'
            elif compartment == 'CCO-OUT':
                # mol_cc = mol_cc.in_compartment('p')
                compartment = 'p'
                
            rxn_dict[mol_cc] =  1
            stoich_loop_dict[f'{cid}[{compartment}]']  = 1
    
    rxns_dict[name] = Reaction(rxn_dict)
    stoich_dict[name] = stoich_loop_dict
    regulation_dict[name] = regulation_loop_dict

# (standard_dg_prime, dg_uncertainty) = cc.standard_dg_prime_multi(list(rxns_dict.values()), uncertainty_representation="cov")
# R = 0.008314 # kJ/mol*K
# T = 298.15 # K
# dG = standard_dg_prime._magnitude
# 
# K_eq = -dG/(R*T)

rxns_dict

In [12]:
stoich_dict

# Drop water and protons 

In [13]:
# drop water and protons from dict
for rxn in stoich_dict:
    stoich_dict[rxn] = {k: v for k, v in stoich_dict[rxn].items() if 'WATER' not in k and 'PROTON' not in k}
    
stoich_dict

# Calculating set of kinetic parameters with given equilibrium constants and arbitrary fluxes

In [90]:
Sd = pd.DataFrame(stoich_dict).fillna(0).astype(np.int8)
# Sd = Sd.iloc[0:7, 0:2]

n_rxn = Sd.shape[1]
n_met = Sd.shape[0]

Sd

# Sd["6PFRUCTPHOS-RXN"] = Sd["6PFRUCTPHOS-RXN"] # * 2
Sd

In [91]:
Sr_min = pd.DataFrame(regulation_dict).fillna(0).astype(np.int8)

Sr = Sd.copy()
Sr.loc[:,:] = 0
Sr.loc[Sd.index.intersection(Sr_min.index), :] = Sr_min.loc[Sd.index.intersection(Sr_min.index), :]
Sr

In [92]:
# estimate flows
# ZnuBC has 125 copies per cell normally. Grows to about 600 copies per cell in zinc starvation.
# ZntA has 300 copies per cell normally. Falls to zero in zinc starvation.
# dynamic equilibrium flows are as follows in counts:
# Zn_in = 400 atoms per second
# Zn_out = 200 atoms per second
# Zn_prot = 200 atoms per second

# convert flows to molar/second
cell_volume = 3e-15 # L
Zn_in = 400 / (6.022e23 * cell_volume)
Zn_out = 200 / (6.022e23 * cell_volume)
Zn_prot = 200 / (6.022e23 * cell_volume)

In [93]:
vE

In [94]:
# K_eq = np.log(keq)
K_eq = np.array([8, 8])
K_eq_mod = K_eq[:, np.newaxis].T
vE = np.array([[Zn_in, Zn_out*2], 
               [Zn_in, Zn_out], 
               [Zn_in, Zn_out*4],
               [1.2*Zn_in, 1.2*Zn_out],
               [Zn_in, Zn_out],
               [0.8*Zn_in, 0.8*Zn_out],
               [Zn_in, Zn_out],
               [Zn_in, Zn_out],
               [Zn_in, Zn_out],])
# vE[:, 0] = vE[:, 0] / 2

n_flux_set = vE.shape[0]

# K_eq[vE < 0] = 1/K_eq[vE < 0]

lvE = np.log(np.abs(vE))
pd.DataFrame(np.concatenate([K_eq_mod, vE, np.sign(vE)]), columns=Sd.columns,
             index=["$K_{eq}$", ] + [f"Flux set {i}" for i in range(n_flux_set)] + [f"Sign set {i}" for i in range(n_flux_set)])

In [95]:
[Zn_in, Zn_out],[Zn_in, Zn_out],# set up variables

S_mol = np.array(Sd)
S = np.sign(S_mol) #
S_s = -np.copy(S) # reverse neg sign
S_p = np.copy(S)
S_s[S > 0] = 0 # zeros products
S_p[S < 0] = 0 # zeros substrates
S_i = np.copy(np.array(Sr) == -1) # reaction direction does not matter
S_a = np.copy(np.array(Sr) == 1)


S_s_nz = np.array(S_s.nonzero())
S_p_nz = np.array(S_p.nonzero())
S_i_nz = np.array(S_i.nonzero())
S_a_nz = np.array(S_a.nonzero())
S_s_mol = np.abs(S_mol)[S_s.nonzero()]
S_p_mol = np.abs(S_mol)[S_p.nonzero()]

# TODO Refactor all the below lines as one liners
# first coordinate, e.g. metabolites w nonzero substrate/product coeff across all reactions. also works as substrate indices.
met_s_nz = S_s_nz[0, :]
met_p_nz = S_p_nz[0, :]
met_i_nz = S_i_nz[0, :]
met_a_nz = S_a_nz[0, :]

# second coordinate, e.g. reactions indices for those concentrations. works to index substrates as well.
rxn_s_nz = S_s_nz[1, :]
rxn_p_nz = S_p_nz[1, :]
rxn_i_nz = S_i_nz[1, :]
rxn_a_nz = S_a_nz[1, :]

# one dim is always 2
n_Km_s = np.max(met_s_nz.shape)
n_Km_p = np.max(met_p_nz.shape)
n_Km_i = np.max(met_i_nz.shape)
n_Km_a = np.max(met_a_nz.shape)

c = cp.Variable([n_met, n_flux_set])
Km_s = cp.Variable(n_Km_s)
Km_p = cp.Variable(n_Km_p)
Km_i = cp.Variable(n_Km_i) if n_Km_i else None
Km_a = cp.Variable(n_Km_a) if n_Km_a else None

cfwd = cp.Variable(n_rxn)
crev = cp.Variable(n_rxn)

# define y vecs
y_s_t = []
y_p_t = []
y_i_t = []
y_a_t = []

# define Km positions by nonzero S matrix concentrations. Activation is reverse val of inhibition.
# TODO Add molecularity here.
for i in range(n_flux_set):
    y_s_t.append(cp.multiply(S_s_mol, c[met_s_nz, i] - Km_s))
    y_p_t.append(cp.multiply(S_p_mol, c[met_p_nz, i] - Km_p))
    y_i_t.append(c[met_i_nz, i] - Km_i if n_Km_i else None)
    y_a_t.append(-(c[met_a_nz, i] - Km_a) if n_Km_a else None)

y_s = cp.vstack(y_s_t)
y_p = cp.vstack(y_p_t)
y_i = cp.vstack(y_i_t)
y_a = cp.vstack(y_a_t)

# saturation stacks
y_f_vec = [y_s]
y_r_vec = [y_p]
if n_Km_i:
    y_f_vec.append(y_i)
    y_r_vec.append(y_i)
if n_Km_a:
    y_f_vec.append(y_a)
    y_r_vec.append(y_a)

y_f = cp.hstack(y_f_vec)
y_r = cp.hstack(y_r_vec)

In [96]:
# number of saturation terms for sub, prod
S_s_comb = np.concatenate((S_s, S_i, S_a), axis=0)
S_p_comb = np.concatenate((S_p, S_i, S_a), axis=0)
n_alpha = np.sum(np.power(2, np.sign(S_s_comb).sum(axis=0)) - 1)
n_beta = np.sum(np.power(2, np.sign(S_p_comb).sum(axis=0)) - 1)

# saturation matrix setup, first sub, then inhib, then act.
C_alpha = np.zeros([n_alpha, len(met_s_nz) + len(met_i_nz) + len(met_a_nz)])
C_beta = np.zeros([n_beta, len(met_p_nz) + len(met_i_nz) + len(met_a_nz)])

# to separate different reactions saturation terms to their individual reaction equations.
d_alpha = np.zeros(n_alpha, dtype=np.int8)
d_beta = np.zeros(n_beta, dtype=np.int8)


idx = 0

for i in range(n_rxn):

    # pick one reaction at a time (get substrate indicies)
    #idx_cur_rxn = rxn_s_nz == i
    # TODO This does not properly multiply by molecularity. Alternatively, generate C_alpha and
    # TODO beta without molecularity (first ==1) and then multiply by molecularity in the end.
    idx_cur_rxn = np.concatenate((rxn_s_nz == i, rxn_i_nz == i, rxn_a_nz == i))

    # generates all binary permutations minus the first one since that would result in -1
    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]

    r, _ = sat_perm.shape

    # replace zeros with saturation matrix
    C_alpha[idx:(idx+r), idx_cur_rxn] = sat_perm
    d_alpha[idx:(idx+r)] = i

    idx += r # add row #

idx = 0

for i in range(n_rxn):
    idx_cur_rxn = np.concatenate((rxn_p_nz == i, rxn_i_nz == i, rxn_a_nz == i))

    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]

    r, _ = sat_perm.shape

    C_beta[idx:(idx+r), idx_cur_rxn] = sat_perm
    d_beta[idx:(idx+r)] = i

    idx += r # add row #

In [97]:
C_alpha

In [98]:
n_lse_terms = np.max(np.power(2, S_s.sum(axis=0)) +  np.power(2, S_p.sum(axis=0)) - 2)
LSE_expr = []
denom_expr = []

sign = np.sign(vE)
lvE = np.log(sign * vE)

for j in range(n_flux_set):
    for i in range(n_rxn):
        # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)

        n_term_s = np.sum(d_alpha == i)
        n_term_p = np.sum(d_beta == i)
        n_term = n_term_s + n_term_p

        Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
        S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries

        Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
        S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]

        if sign[j, i] == 1:
            LSE_expr.append(cp.hstack( [
                                         lvE[j, i] + (C_alpha @ cp.vec(y_f[j, :]))[d_alpha == i]
                                            - cp.multiply(np.ones(n_term_s), - S.T[i, S_s_idx] @ cp.vec(y_s[j, Km_s_idx])) - cfwd[i],
                                         lvE[j, i] + (C_beta @ cp.vec(y_r[j, :]))[d_beta == i]
                                            - cp.multiply(np.ones(n_term_p), - S.T[i, S_s_idx] @ cp.vec(y_s[j, Km_s_idx])) - cfwd[i],

                                         lvE[j, i] + 0 - cp.multiply(np.ones(1), -S.T[i, S_s_idx] @ cp.vec(y_s[j, Km_s_idx]))  - cfwd[i],

                                         cp.multiply(np.ones(1), S.T[i, S_p_idx] @ cp.vec(y_p[j, Km_p_idx]))  + crev[i]
                                            - cp.multiply(np.ones(1), -S.T[i, S_s_idx] @ cp.vec(y_s[j, Km_s_idx]))  - cfwd[i],

                                       ]
                                     )
                           )  # remove +1 here, could also have cfwd outside objec.

            denom_expr.append(cp.multiply(np.ones(1), -S.T[i, S_s_idx] @ cp.vec(y_s[j, Km_s_idx])) + cfwd[i],)


        # keep saturation term the same, switch around fwd and rev terms. flip all signs with S matrix since it's signed.
        if sign[j, i] == -1:
            LSE_expr.append(cp.hstack( [ lvE[j, i] + (C_alpha @ cp.vec(y_f[j, :]))[d_alpha == i]
                                            - cp.multiply(np.ones(n_term_s), S.T[i, S_p_idx] @ cp.vec(y_p[j, Km_p_idx])) - crev[i],

                                         lvE[j, i] + (C_beta @ cp.vec(y_r[j, :]))[d_beta == i]
                                            - cp.multiply(np.ones(n_term_p), S.T[i, S_p_idx] @ cp.vec(y_p[j, Km_p_idx])) - crev[i],

                                         lvE[j, i] + 0 - cp.multiply(np.ones(1), S.T[i, S_p_idx] @ cp.vec(y_p[j, Km_p_idx])) - crev[i],

                                         cp.multiply(np.ones(1), - S.T[i, S_s_idx] @ cp.vec(y_s[j, Km_s_idx]))  + cfwd[i]
                                            - cp.multiply(np.ones(1), S.T[i, S_p_idx] @ cp.vec(y_p[j, Km_p_idx])) - crev[i],

                                       ]
                                     )
                           )

            denom_expr.append(cp.multiply(np.ones(1), S.T[i, S_p_idx] @ cp.vec(y_p[j, Km_p_idx])) + crev[i])


#LSE_expr = cp.vstack(LSE_expr)
LSE_expr

In [99]:
l = 0.001
e = 0.01
f = 0.01
g = 1
reg =  cp.sum(cp.hstack([cfwd, crev, cp.vec(c)])) + cp.sum(cp.hstack([-Km_s, -Km_p])) # regularization
reg2 = (cp.norm1(cp.hstack([cfwd, crev, cp.vec(c)])) 
        #+ cp.norm1(cp.hstack([-Km_s, -Km_p]) - 4)
        ) # regularization
reg3 = cp.sum(cp.huber(cp.hstack([-Km_s, -Km_p]) - 4,3  )) # issue with matrix

# if y_s is larger than 3 or smaller than -3, add penalty
reg4 = cp.sum(cp.max(cp.abs(cp.hstack([y_s, y_p])) - 3, 0))

if n_Km_i:
    reg += cp.sum(cp.hstack([-Km_i]))
if n_Km_a:
    reg += cp.sum(cp.hstack([-Km_a]))
#reg3 = cp.norm1(cp.hstack([y_s, y_p])) # take a look at this

loss = 0
# for i in range(len(LSE_expr)):
#     loss += cp.norm1(cp.pos(cp.log_sum_exp(LSE_expr[i])))
for i in range(len(denom_expr)):
    loss += 1 * denom_expr[i]
loss += l * reg
# loss += e * reg2
# loss += f * reg3
loss += g * reg4

In [100]:
cp.max(cp.abs(cp.hstack([y_s, y_p])) - 3, 0)

In [101]:
# concentration estimates
# Zn_conc_in = 1e-12 M
# Zn_conc_out = 1e-6 M
# ATP = 5e-3 M
# ADP = 5e-4 M
# Pi = 5e-4 M

# km measured
# ZntA_Zn_in = 1e-12 M

list(Sd.index)

In [102]:
concentrations = {
    'ZN+2[p]': [1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 0.9e-6, 1.1e-6, 0.95e-6, ],
    'ZN+2[c]': [1e-12, 0.5e-12, 2e-12,1e-12,1e-12,1e-12,0.9e-12,1.1e-12,0.95e-12],
    'ATP[c]': [5e-3, 5e-3, 5e-3,5e-3, 5e-3, 5e-3,5.1e-3, 5.05e-3, 4.95e-3],
    'ADP[c]': [5e-4, 5e-4, 5e-4,5e-4, 5e-4, 5e-4,5.1e-4, 5.05e-4, 4.95e-4],
    'Pi[c]': [5e-4, 5e-4, 5e-4,5e-4, 5e-4, 5e-4,5.1e-4, 4.95e-4, 5.1e-4],
}



In [103]:
constr = [cp.hstack([cfwd, crev, cp.vec(c), Km_s, Km_p]) >= -50,
          cp.hstack([cfwd, crev, cp.vec(c), Km_s, Km_p]) <= 6,
          ]

if n_Km_i:
    constr.extend([Km_i >= -12, Km_i <= 12])
if n_Km_a:
    constr.extend([Km_a >= -12, Km_a <= 12])

# for now, just set concentrations as a constraint
for i, met in enumerate(Sd.index):
    if met in concentrations:
        constr.extend([c[i, 0] == np.log(concentrations[met][0])])
        constr.extend([c[i, 1] == np.log(concentrations[met][1])])
        

for i in range(len(LSE_expr)):
    constr.extend([cp.pos(cp.log_sum_exp(LSE_expr[i])) <= 0])
# set Kms as a constraint

In [104]:
haldane = []
fwd_flux = []

for i, r in enumerate(S.T):
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries

    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]

    haldane.append(K_eq[i] == cfwd[i] - crev[i] + r[S_p_idx] @ Km_p[Km_p_idx] - (-r[S_s_idx]) @ Km_s[Km_s_idx])

for j in range(n_flux_set):
    for i, r in enumerate(S.T):
        Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
        S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries

        Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
        S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]

        if sign[j, i] == 1:
            fwd_flux.append(cfwd[i] + (-r[S_s_idx]) @ cp.vec(y_s[j, Km_s_idx]) - (crev[i] + r[S_p_idx] @ cp.vec(y_p[j, Km_p_idx]))  >= 0)  # add minus since s matrix has minus

        if sign[j, i] == -1:
            fwd_flux.append(cfwd[i] + (-r[S_s_idx]) @ cp.vec(y_s[j, Km_s_idx]) - (crev[i] + r[S_p_idx] @ cp.vec(y_p[j, Km_p_idx]))  <= 0)  # add minus since s matrix has minus

    constr.extend([cp.multiply(S.T @ cp.vec(c[:, j]), sign[j, :])  <= cp.multiply(K_eq, sign[j, :])])

# constr.extend(haldane)
constr.extend(fwd_flux)

In [105]:
p = cp.Problem(cp.Minimize(loss), constr)
p.solve(verbose=True, solver=cp.CLARABEL)

In [106]:
print('Substrate Km:', [f'{val:.6f}' for val in np.exp(Km_s.value)])
print('Product Km:', [f'{val:.3f}' for val in np.exp(Km_p.value)])
print('Fwd kcat:', [f'{val:.3f}' for val in np.exp(cfwd.value)])
print('Rev kcat:', [f'{val:.3f}' for val in np.exp(crev.value)])

concs = np.exp(c.value).T
for row in concs:
    print('Concentration:', [f'{val:.12f}' for val in row])

if n_Km_i:
    print('Inhibition Km:', [f'{val:.7f}' for val in np.exp(Km_i.value)])
if n_Km_a:
    print('Activation Km:', [f'{val:.7f}' for val in np.exp(Km_a.value)])

In [107]:
print('reaction 1, ZnuBC Out -> In, no water, proton')
print('List of substrates: ', list(Sd.index[Sd.iloc[:, 0] < 0]))
print('Substrate Km:', [f'{val:.4f}' for val in Km_s.value[0:2]])
print('List of products: ', list(Sd.index[Sd.iloc[:, 0] > 0]))
print('Product Km:', [f'{val:.4f}' for val in Km_p.value[0:3]])

print('Fwd kcat:', [f'{val:.4f}' for val in cfwd.value][0])
print('Rev kcat:', [f'{val:.4f}' for val in crev.value][0])


print('reaction 2, ZntA in -> out, no water, proton')
print('List of substrates: ', list(Sd.index[Sd.iloc[:, 1] < 0]))
print('Substrate Km:', [f'{val:.4f}' for val in Km_s.value[2:4]])
print('List of products: ', list(Sd.index[Sd.iloc[:, 1] > 0]))
print('Product Km:', [f'{val:.4f}' for val in Km_p.value[3:6]])

print('Fwd kcat:', [f'{val:.4f}' for val in cfwd.value][1])
print('Rev kcat:', [f'{val:.4f}' for val in crev.value][1])

# concentrations
print('Molecules: ', list(Sd.index))
print('Concentration:', [f'{val:.4f}' for val in c.value[:, 0]])

In [108]:
y_p.value

In [109]:
for v in LSE_expr:
    #print(v.value)
    print(logsumexp(v.value))

Perfect? Wow.

# Check flux reconstruction with inhibition/activation

In [110]:
reconstructed_vE = np.zeros(vE.shape)

for j in range(n_flux_set):
    sat_expr = []
    fwd_sat = np.zeros(n_rxn)
    back_sat = np.zeros(n_rxn)
    sat = np.zeros(n_rxn)

    for i in range(n_rxn):
        # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)
        n_term_s = np.sum(d_alpha == i)
        n_term_p = np.sum(d_beta == i)
        n_term = n_term_s + n_term_p


        Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
        S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries

        Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
        S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]

        #S_s_idx = S_s_nz[0, S_s_nz[1, :] == i]

        sat_expr.append(           [ (C_alpha @ y_f.value[j, :].flatten())[d_alpha == i] ,
                                     (C_beta @ y_r.value[j, :].flatten())[d_beta == i],
                                     0,
                                     #-1*np.ones(n_lse_terms - n_term + 1)
                                   ]
                       )
        fwd_sat[i] = (np.exp(-S.T[i, S_s_idx] @ y_s.value[j, Km_s_idx].flatten())) # + cfwd.value[i]
        back_sat[i] = (np.exp(S.T[i, S_p_idx] @ y_p.value[j, Km_p_idx].flatten())) # + cfwd.value[i]



    for i, rxn in enumerate(sat_expr):
        s = 0

        for term in rxn:
            s += np.sum(np.exp(term))

        sat[i] = (s)

    reconstr = np.exp(cfwd.value) * fwd_sat/sat - np.exp(crev.value) * back_sat/sat
    print(reconstr)
    reconstructed_vE[j, :] = reconstr

In [111]:
df_vE = pd.DataFrame(vE, columns=Sd.columns, index=["Flux set 1", "Flux set 2","Flux set 3"]).melt(ignore_index=False).reset_index(drop=False)
df_vE["kind"] = "Actual flux"
df_recon = pd.DataFrame(reconstructed_vE, columns=Sd.columns, index=["Flux set 1", "Flux set 2","Flux set 3"]).melt(ignore_index=False).reset_index(drop=False)
df_recon["kind"] = "Reconstructed flux"

df_reconstr_comp = pd.concat([df_vE, df_recon]).reset_index(drop=True)
df_reconstr_comp

In [112]:
sns.catplot(x="index", y="value", hue="kind", col="variable", kind="bar", data=df_reconstr_comp, height=5, aspect=1, col_wrap=2)
plt.savefig("flux_set_comparison.svg")

In [113]:
reconstructed_vE = np.zeros(vE.shape)

for j in range(n_flux_set):
    sat_expr = []
    fwd_sat = np.zeros(n_rxn)
    back_sat = np.zeros(n_rxn)
    sat = np.zeros(n_rxn)

    for i in range(n_rxn):
        # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)


        Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
        S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries

        Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
        S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]

        sat_expr.append(           [ (C_alpha @ y_f.value[j, :].flatten())[d_alpha == i] ,
                                     (C_beta @ y_r.value[j, :].flatten())[d_beta == i],
                                     0,
                                     #-1*np.ones(n_lse_terms - n_term + 1)
                                   ]
                       )
        fwd_sat[i] = (np.exp(-S.T[i, S_s_idx] @ y_s.value[j, Km_s_idx].flatten())) 
        back_sat[i] = (np.exp(S.T[i, S_p_idx] @ y_p.value[j, Km_p_idx].flatten()))



    for i, rxn in enumerate(sat_expr):
        s = 0

        for term in rxn:
            s += np.sum(np.exp(term))

        sat[i] = (s)

    reconstr = np.exp(cfwd.value) * fwd_sat/sat - np.exp(crev.value) * back_sat/sat
    print(reconstr)
    reconstructed_vE[j, :] = reconstr

In [114]:
dmdt_ext = S @ reconstructed_vE.T
dmdt_ext

# Simulate dynamic response

In [115]:
c.value

In [81]:
Sd.index

In [117]:
np.log(np.array([1e-6, 5e-3, 1e-12, 5e-4, 5e-4,]))

In [282]:
Cc = []
Dmdt = []
Vv = []

for start in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    
    
    c = np.log(np.array([1e-6, 5e-3, 1e-12 * start, 5e-4, 5e-4,]))
    
    C = []
    D = []
    V = []
    
    for t in range(10000):
        
        v = np.zeros(n_rxn)
    
        for i in range(n_rxn):
            y_s_t = []
            y_p_t = []
            sat_expr = []
            
            j = 0
                
            Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
            S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
            
            
            Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
            S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
            
            y_s_t.append(np.multiply(S_s_mol, c[met_s_nz] - Km_s.value))
            y_p_t.append(np.multiply(S_p_mol, c[met_p_nz] - Km_p.value))
            
            y_s = np.vstack(y_s_t)
            y_p = np.vstack(y_p_t)
            
            y_f_vec = [y_s]
            y_r_vec = [y_p]
            
            y_f = np.hstack(y_f_vec)
            y_r = np.hstack(y_r_vec)
            
            sat_expr = list((C_alpha @ y_f[j, :].flatten())[d_alpha == i]) + list((C_beta @ y_r[j, :].flatten())[d_beta == i]) + [0]
            sat_exp = np.sum(np.exp(sat_expr))
            
            fwd_sat = (np.exp(-S.T[i, S_s_idx] @ y_s[j, Km_s_idx].flatten())) 
            back_sat = (np.exp(S.T[i, S_p_idx] @ y_p[j, Km_p_idx].flatten()))
            
            v[i] = np.exp(cfwd.value[i]) * fwd_sat/sat_exp - np.exp(crev.value[i]) * back_sat/sat_exp
            
        
        v[1] += 0.8 * v[0]
        dmdt = S @ np.array(v)
        
        # add noise
        noise = np.random.normal(0, 5*10**(-16))
        
        c[2] = np.log(np.max([np.exp(c[2]) + dmdt[2] * 0.0000001 + noise,10**(-18) ]   )) 
        
        C.append(c[2])
        D.append(dmdt[2])
        V.append(v[0])
        
    Cc.append(C)
    Dmdt.append(D)
    Vv.append(V)

In [283]:
plt.plot(np.array(Cc).T )